In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
face_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_eye.xml')


## Defining a function to crop the given proper image 

In [3]:
def get_cropped_image_if_2_eyes(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                return roi_color

## Code to crop all the images and save them

In [4]:
path_to_data = "./dataset/full_images/"
path_to_cr_data = "./dataset/cropped/"

In [5]:
import os
import shutil
if os.path.exists(path_to_cr_data):
     shutil.rmtree(path_to_cr_data)

In [6]:
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)
img_dirs


['./dataset/full_images/other', './dataset/full_images/person']

In [7]:
os.mkdir(path_to_cr_data)

In [8]:
# for img_dir in img_dirs:
#     count = 1
#     individual_name = img_dir.split('/')[-1]
#     print(individual_name)

In [8]:
individual_file_names_dict = {}
for img_dir in img_dirs:
    count = 1
    individual_name = img_dir.split('/')[-1]
    
    individual_file_names_dict[individual_name] = []
    
    cropped_individual_folder = path_to_cr_data + individual_name
    os.mkdir(cropped_individual_folder)
    
    for entry in os.scandir(img_dir):
        roi_color = get_cropped_image_if_2_eyes(entry.path)
        if roi_color is not None:          
            cropped_file_name = "image_cropped" + str(count) + ".png"
            cropped_file_path = cropped_individual_folder + '/' + cropped_file_name 
            
            cv2.imwrite(cropped_file_path, roi_color)
            individual_file_names_dict[individual_name].append(cropped_file_path)
            count += 1    

In [9]:
individual_to_number_dict = {"person" : 1, "other" : 0}

# Data Preprocessing

## Wavelet Transformation to extract features from the cropped images

In [10]:
import pywt

In [11]:
def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

## Images in cropped folder can be used for model training. We will use these raw images along with wavelet transformed images to train our classifier. Let's prepare X and y now

In [12]:
X, y = [], []

for cropped_dir in os.scandir(path_to_cr_data):
    individual_name = cropped_dir.path.split('/')[-1]
    
    for training_image in os.scandir(cropped_dir):
        img = cv2.imread(training_image.path)
        scalled_raw_img = cv2.resize(img, (32, 32))
        img_har = w2d(img,'db1',5)
        scalled_img_har = cv2.resize(img_har, (32, 32))
        combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
        X.append(combined_img)
        y.append(individual_to_number_dict[individual_name]) 


In [13]:
X

[array([[ 12],
        [ 12],
        [ 25],
        ...,
        [131],
        [  7],
        [248]], dtype=uint8),
 array([[  6],
        [ 12],
        [ 24],
        ...,
        [232],
        [ 74],
        [225]], dtype=uint8),
 array([[  8],
        [  4],
        [  5],
        ...,
        [ 14],
        [246],
        [248]], dtype=uint8),
 array([[ 25],
        [ 23],
        [ 37],
        ...,
        [230],
        [ 23],
        [ 69]], dtype=uint8),
 array([[ 47],
        [ 31],
        [118],
        ...,
        [254],
        [  1],
        [  0]], dtype=uint8),
 array([[ 1],
        [12],
        [30],
        ...,
        [84],
        [ 5],
        [ 1]], dtype=uint8),
 array([[110],
        [133],
        [181],
        ...,
        [  4],
        [  4],
        [127]], dtype=uint8),
 array([[224],
        [226],
        [204],
        ...,
        [  5],
        [133],
        [ 51]], dtype=uint8),
 array([[ 11],
        [ 10],
        [ 21],
        ...,
    

In [14]:
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [15]:
y_df = pd.DataFrame(y)
y_df.value_counts()

1    735
0    342
dtype: int64

In [16]:
X = np.array(X).reshape(len(X),4096).astype(float)
X.shape

(1077, 4096)

In [147]:
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [18]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto',probability=True),
        'params' : {
            'svc__C': [1,10,100,1000],
            'svc__kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(random_state=42),
        'params' : {
            'randomforestclassifier__n_estimators': [5,10,20],
            'randomforestclassifier__max_features': ['auto', 'sqrt', 'log2'],
            'randomforestclassifier__bootstrap': [True],
            'randomforestclassifier__min_samples_leaf': [1,2,3, 4, 5],
            'randomforestclassifier__min_samples_split': [3, 4, 6, 8, 10, 12]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(multi_class='auto'),
        'params': {
#             'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear'],
            'logisticregression__solver': ['liblinear'],
            'logisticregression__C': [1,5,10]
#             'logisticregression__penalty': [None,'l1','l2','elasticnet']
        }
    },
    'kneighborsclassifier': {
        'model': KNeighborsClassifier(),
        'params': {
            'kneighborsclassifier__n_neighbors': list(range(1,30))
#             'kneighborsclassifier__p': [1,2],
#             'kneighborsclassifier__leaf_size': list(range(1,50))
        }
    },
    'adaboostclassifier': {
        'model': AdaBoostClassifier(),
        'params': {
            'adaboostclassifier__n_estimators': [5,10,20]
        }
    },
    'gaussiannb': {
        'model': GaussianNB(),
        'params': {
            'gaussiannb__var_smoothing': np.logspace(0,-9, num=100)
        }
    }

    
}

In [19]:
scores = []
best_estimators = {}

for algo, mp in model_params.items():
    pipe = make_pipeline(QuantileTransformer(n_quantiles=320), mp['model'])
    clf =  GridSearchCV(pipe, mp['params'], cv=5, return_train_score=False)
    clf.fit(X,y)
    scores.append({
        'model': algo,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimators[algo] = clf.best_estimator_
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and Extra

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and Extra

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and Extra

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and Extra

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and Extra

...

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and Extra

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and Extra

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and Extra

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and Extra

,model,best_score,best_params
0,svm,0.996279,"{'svc__C': 10, 'svc__kernel': 'rbf'}"
1,random_forest,0.999070,"{'randomforestclassifier__bootstrap': True, 'r..."
2,logistic_regression,0.988846,"{'logisticregression__C': 1, 'logisticregressi..."
3,kneighborsclassifier,0.987911,{'kneighborsclassifier__n_neighbors': 2}
4,adaboostclassifier,0.971167,{'adaboostclassifier__n_estimators': 20}
5,gaussiannb,0.974901,{'gaussiannb__var_smoothing': 0.0657933224657568}


In [20]:
best_estimators

{'svm': Pipeline(steps=[('quantiletransformer', QuantileTransformer(n_quantiles=320)),
                 ('svc', SVC(C=10, gamma='auto', probability=True))]),
 'random_forest': Pipeline(steps=[('quantiletransformer', QuantileTransformer(n_quantiles=320)),
                 ('randomforestclassifier',
                  RandomForestClassifier(max_features='log2', min_samples_leaf=5,
                                         min_samples_split=3, n_estimators=20,
                                         random_state=42))]),
 'logistic_regression': Pipeline(steps=[('quantiletransformer', QuantileTransformer(n_quantiles=320)),
                 ('logisticregression',
                  LogisticRegression(C=1, solver='liblinear'))]),
 'kneighborsclassifier': Pipeline(steps=[('quantiletransformer', QuantileTransformer(n_quantiles=320)),
                 ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=2))]),
 'adaboostclassifier': Pipeline(steps=[('quantiletransformer', QuantileTransformer(

In [21]:
df['best_params'][1]

{'randomforestclassifier__bootstrap': True,
 'randomforestclassifier__max_features': 'log2',
 'randomforestclassifier__min_samples_leaf': 5,
 'randomforestclassifier__min_samples_split': 3,
 'randomforestclassifier__n_estimators': 20}

In [22]:
clf = best_estimators['random_forest']

In [54]:
clf2 = best_estimators['logistic_regression']

In [131]:
clf2

Pipeline(steps=[('quantiletransformer', QuantileTransformer(n_quantiles=320)),
                ('logisticregression',
                 LogisticRegression(C=1, solver='liblinear'))])

In [162]:
path_to_data_test = "./images_to_classified/"
path_to_cr_data_test = "./images_to_classified_cr/"

In [163]:
if os.path.exists(path_to_cr_data_test):
    shutil.rmtree(path_to_cr_data_test)
os.mkdir(path_to_cr_data_test)

In [164]:
count = 1    
for entry in os.scandir(path_to_data_test):
    roi_color = get_cropped_image_if_2_eyes(entry.path)
    if roi_color is not None:          
        cropped_file_name = "image_cropped_test" + str(count) + ".png"
        cropped_file_path = path_to_cr_data_test + cropped_file_name 
            
        cv2.imwrite(cropped_file_path, roi_color)
        count += 1    

In [173]:
X_test, y_test = [], []

path2 = "./copy/"
for test_image in os.scandir(path2):
    img = cv2.imread(test_image.path)
    scalled_raw_img = cv2.resize(img, (32, 32))
    img_har = w2d(img,'db1',5)
    scalled_img_har = cv2.resize(img_har, (32, 32))
    combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
    X_test.append(combined_img)
    y_test.append(0) 


In [174]:
X_test = np.array(X_test).reshape(len(X_test),4096).astype(float)
X_test.shape

(123, 4096)

In [116]:
clf.score(X_test,y_test)

0.016260162601626018

In [117]:
clf.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [175]:
clf2.score(X_test,y_test)

0.0

In [176]:
clf2.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [69]:
clf.score(X,y)

1.0

In [124]:
clf2.score(X,y)

1.0

In [154]:
clf3 = best_estimators['adaboostclassifier']

In [169]:
clf3.score(X_test,y_test)

0.45528455284552843

In [170]:
clf3.predict(X_test)

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [157]:
clf5 = best_estimators['kneighborsclassifier']

In [158]:
clf5.score(X_test,y_test)

0.0

In [159]:
clf5.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [44]:
model_params2 = {
    'adaboostclassifier' : {
        'model' : AdaBoostClassifier(),
        'params' : {
            'adaboostclassifier__n_estimators' : [100, 200, 300, 400, 500],
            'adaboostclassifier__random_state' : [42]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(multi_class='auto'),
        'params': {
#             'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear'],
            'logisticregression__solver': ['liblinear'],
            'logisticregression__C': [1,5,10],
            'logisticregression__penalty': ['l1']
        }
    }
}

In [47]:
scores2 = []
best_estimators2 = {}
for algo, mp in model_params2.items():
    pipe4 = make_pipeline(StandardScaler(), mp['model'])
    clf4 =  GridSearchCV(pipe4, mp['params'], cv=10, return_train_score=False)
    clf4.fit(X,y)
    scores2.append({
        'model': algo,
        'best_score': clf4.best_score_,
        'best_params': clf4.best_params_
    })
    best_estimators2[algo] = clf4.best_estimator_
    
df = pd.DataFrame(scores2,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,adaboostclassifier,1.000000,"{'adaboostclassifier__n_estimators': 100, 'ada..."
1,logistic_regression,0.997205,"{'logisticregression__C': 5, 'logisticregressi..."


## h

In [75]:
from sklearn.pipeline import Pipeline
pipe_extra = Pipeline([('scaler', QuantileTransformer(n_quantiles=320)), ('randomforest', RandomForestClassifier(random_state=42, bootstrap=True,
 max_features='auto',
 min_samples_leaf= 1,
 min_samples_split=3,
 n_estimators=10))])
pipe_extra.fit(X,y)
pipe_extra.predict(X_test)

C:\Python3105\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Getting accuracy with extra test_data

In [21]:
# from sklearn.pipeline import Pipeline
# params2 = {'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear'],
#             'logisticregression__C': [1,5,10],
#             'logisticregression__penalty': [None,'l1','l2','elasticnet']}
# pipe2 = Pipeline([('scaler', QuantileTransformer(n_quantiles=320)), ('logisticregression', LogisticRegression())])
# clf2 =  GridSearchCV(pipe2, param_grid=params2, cv=5, return_train_score=False)
# clf2.fit(X,y)
# df_lg = pd.DataFrame({
#     'model': 'logisticregression',
#     'best_score': clf2.best_score_,
#     'best_params': clf2.best_params_
# })
# df_lg

In [22]:
# best_estimator2 = clf2.best_estimator_

In [23]:
# best_estimator2

In [24]:
# test_dir = './test_images/'
# testimages_list = []

# for entry in os.scandir(test_dir):
#     testimages_list.append(entry.path)

In [25]:
# count=1
# path_croppedimgs= []
# for entry in os.scandir(test_dir):
#         roi_color = get_cropped_image_if_2_eyes(entry.path)
#         if roi_color is not None:
#             cropped_folder = test_dir + 'croppedtest_img'
#             if not os.path.exists(cropped_folder):
#                 os.makedirs(cropped_folder)
                
#             cropped_file_name = str(count) + ".png"
#             cropped_file_path = cropped_folder + "/" + cropped_file_name 
            
#             cv2.imwrite(cropped_file_path, roi_color)
#             path_croppedimgs.append(cropped_file_path)
#             count += 1    

In [26]:
# cropped_img2 = []

# for image in os.scandir(cropped_folder):
#     cropped_img2.append(image.path)
    

In [27]:
# from sklearn.metrics import confusion_matrix

# X_test_img = []

# for training_image in cropped_img2:
#         img = cv2.imread(training_image)
#         scalled_raw_img = cv2.resize(img, (32, 32))
#         img_har = w2d(img,'db1',5)
#         scalled_img_har = cv2.resize(img_har, (32, 32))
#         combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
#         X_test_img.append(combined_img)
        
# X_test_img = np.array(X_test_img).reshape(len(cropped_img2),4096).astype(float)


In [28]:
# X_test_img[0]

In [29]:
# class_dict

In [30]:
# y_test_img = np.array([2,2,2,2,2,2,5,5,5,5,5,3,3,3,0,3,3,3,3,6,6,6,6,6,6,1,1,1,1,1,1,1,1,1,0,0,2,2,2,2,2,2,2,2,2,6,6])

In [31]:
# len(y_test_img)

In [32]:
# y_test_img

In [33]:
# res_extra = best_estimator2.predict(X_test_img)

In [34]:
# res_extra

In [35]:
# best_estimator2.score(X_test_img,y_test_img)

In [36]:
# fpr, tpr, thresholds = metrics.roc_curve(y_test_img, res_extra, pos_label=2)
# metrics.auc(fpr, tpr)

## Getting accuracy with train-test split 

In [134]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [139]:
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, y, random_state=0)

lnreg = LogisticRegression(C=1, penalty='l1', solver='liblinear', random_state=42)
pipe_tt = Pipeline([('scaler', StandardScaler()), ('logisticregression', lnreg)])
pipe_tt.fit(X_train_split, y_train_split)

Pipeline(steps=[('scaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(C=1, penalty='l1', random_state=42,
                                    solver='liblinear'))])

In [177]:
pipe_tt.score(X_test_split, y_test_split)

0.9962962962962963

In [141]:
pipe_tt.predict(X_test_split)

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1])

In [41]:
# np.array(y_test)

In [178]:
pipe_tt.score(X_test, y_test)

0.06504065040650407

In [179]:
pipe_tt.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [171]:
X_train_split2, X_test_split2, y_train_split2, y_test_split2 = train_test_split(X, y, random_state=42)

xgbst = XGBClassifier(max_depth=3, min_child_weight=3, scale_pos_weight=3)
pipe_tt2 = Pipeline([('scaler', StandardScaler()), ('xgbclassifier', xgbst)])
pipe_tt2.fit(X_train_split, y_train_split)

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=3, max_leaves=0, min_child_weight=3,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [180]:
pipe_tt2.score(X_test, y_test)

0.4634146341463415

In [181]:
pipe_tt2.predict(X_test)

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [45]:
# fpr, tpr, thresholds = metrics.roc_curve(y_test, res_tt, pos_label=2)
# metrics.auc(fpr, tpr)

## Save the trained model

In [46]:
# !pip install joblib
# import joblib 
# # Save the model as a pickle in a file 
# joblib.dump(best_estimator2, 'saved_model.pkl') 

## Save class dictionary

In [47]:
# import json
# with open("class_dictionary.json","w") as f:
#     f.write(json.dumps(class_dict))